<a href="https://colab.research.google.com/github/adithyamauryakr/pytorchtutorials/blob/main/graph_nn_stuff/graph_attension_layer_gat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.7 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class GATLayer(nn.Module):
  def _init__(self):
    super(GATLayer, self).__init__()

  def forward(self, input, adj):
    print("")

In [5]:
in_features = 5
out_features = 2
nb_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
nn.init.xavier_uniform_(W.data, gain=1.414)

input = torch.rand(nb_nodes, in_features)

h = torch.mm(input, W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [6]:
a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
nn.init.xavier_uniform_(a.data, gain=1.414)
print(a.shape)

leakyrelu = nn.LeakyReLU(0.2)

torch.Size([4, 1])


In [7]:
a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * out_features)

In [8]:
e = leakyrelu(torch.matmul(a_input, a).squeeze(2))

In [9]:
print(a_input.shape)
print(a.shape)

print(torch.matmul(a_input, a).shape)
print('')
print(torch.matmul(a_input, a).squeeze(2).shape)

torch.Size([3, 3, 4])
torch.Size([4, 1])
torch.Size([3, 3, 1])

torch.Size([3, 3])


In [10]:
### Masked attention:
adj = torch.randint(2, (3, 3))

zero_vec = -9e15*torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [11]:
attention = torch.where(adj > 0, e, zero_vec)
print(adj, '\n', e, '\n',zero_vec, attention)

tensor([[0, 0, 1],
        [1, 1, 1],
        [1, 1, 1]]) 
 tensor([[ 0.0411, -0.0252,  1.1363],
        [ 0.2085,  0.0416,  1.3037],
        [-0.1027, -0.1361,  0.5815]], grad_fn=<LeakyReluBackward0>) 
 tensor([[-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15],
        [-9.0000e+15, -9.0000e+15, -9.0000e+15]]) tensor([[-9.0000e+15, -9.0000e+15,  1.1363e+00],
        [ 2.0853e-01,  4.1570e-02,  1.3037e+00],
        [-1.0273e-01, -1.3612e-01,  5.8152e-01]], grad_fn=<WhereBackward0>)


In [12]:
attention = F.softmax(attention, dim=1)
h_prime = torch.matmul(attention, h)

In [13]:
attention

tensor([[0.0000, 0.0000, 1.0000],
        [0.2068, 0.1750, 0.6182],
        [0.2532, 0.2449, 0.5019]], grad_fn=<SoftmaxBackward0>)

In [14]:
h_prime

tensor([[0.8808, 1.0481],
        [0.7153, 0.6656],
        [0.6694, 0.5488]], grad_fn=<MmBackward0>)

In [15]:
print(h_prime, '\n', h)

tensor([[0.8808, 1.0481],
        [0.7153, 0.6656],
        [0.6694, 0.5488]], grad_fn=<MmBackward0>) 
 tensor([[0.3235, 0.0550],
        [0.5939, 0.0361],
        [0.8808, 1.0481]], grad_fn=<MmBackward0>)


In [6]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

name_data = 'Cora'
dataset = Planetoid(root='/tmp/'+name_data, name = name_data)
dataset.transform = T.NormalizeFeatures()

print(f'no. of classes in {name_data}:', dataset.num_classes)
print(f'no. of node features in {name_data}:', dataset.num_node_features)

Processing...


no. of classes in Cora: 7
no. of node features in Cora: 1433


Done!


### Build from scratch:

In [7]:
class GATLayer(nn.Module):

  def __init__(self, in_features, out_features, dropout, alpha, concat=True):
    super(GATLayer, self).__init__()

    self.dropout = dropout
    self.in_features = in_features
    self.out_features = out_features
    self.alpha = alpha # Leaky ReLU with negative input slope, alpha=0.2
    self.concat = concat # concat = True for all layers except the output layer

    # Xavier Initialization of weights
    # alternative: use weights __init__ to apply weights of choice
    self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
    nn.init.xavier_uniform(self.a.data, gain=1.414)

    # leaky ReLU
    self.leakyrelu = nn.LeakyReLU(self.alpha)

  def forward(self, input, adj):

    #linear transformation
    h = torch.mm(input, self.W)
    N = h.size()[0]
    print(N)

    # Attension Mechanism
    a_input = torch.cat([h.repeat(1, N).view(N*N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2*self.out_features)
    e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))

    # Masked Attention
    zero_vec = -9e15*torch.ones_like(e)
    attention = torch.where(adj > 0, e, zero_vec)
    attention = F.softmax(attention, dim=1)
    attention = F.dropout(attention, self.dropout, training=self.training)
    h_prime = torch.matmul(attention, h)

    if self.concat:
      return F.elu(h_prime)
    else:
      return h_prime

### From the torch_geometric module

In [7]:
class GAT(torch.nn.Module):

  def __init__(self):
    super(GAT, self).__init__()

    self.hid = 8
    self.in_head = 8
    self.out_head = 1

    self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
    self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False, heads = self.out_head, dropout=0.6)

  def forward(self,data):
    x, edge_index = data.x, data.edge_index
    x = F.dropout(x, p=0.6, training=self.training)
    x = self.conv1(x, edge_index)
    x = F.elu(x)
    x = F.dropout(x, p=0.6, training=self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.train()
for epoch in range(1000):
  model.train()
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])

  if epoch % 200 == 0:
    print(loss)

  loss.backward()
  optimizer.step()


tensor(1.9444, grad_fn=<NllLossBackward0>)
tensor(0.6018, grad_fn=<NllLossBackward0>)
tensor(0.5977, grad_fn=<NllLossBackward0>)
tensor(0.7075, grad_fn=<NllLossBackward0>)
tensor(0.5331, grad_fn=<NllLossBackward0>)


In [8]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/data.test_mask.sum().item()
print('Accuracy:  {:.4f}'.format(acc))

Accuracy:  0.8180
